<a href="https://colab.research.google.com/github/KatherineMarakhova/RL_attempts/blob/main/last_try1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
import tensorflow as tf
from tensorflow.python.framework import ops
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np

#Список наших бандитов. Бандит №4 наиболее оптимален для выбора.
#bandits = [0.2,0,-0.2,-5]
prices = [1,3,4,2]
bandits = [-0.9, -0.7, -0.8, -0.6]
#ограничение по запасам 35
#ограничения по каждому потребителю
num_bandits = len(bandits)
def pullBandit(bandit):
    #Сгенерировать случайное число
    result = np.random.randn(1)
    if result > bandit:
        #Выигрыш
        return 1
    else:
        #Проигрыш
        return -1

###Агент

In [46]:
ops.reset_default_graph()

#Эти 2 строчки создают feed-forward часть нейросети. Здесь и происходит выбор действия.
weights = tf.Variable(tf.ones([num_bandits]))
chosen_action = tf.argmax(weights,0)

#Следующие 6 строчек устанавливают процедуру обучения. Нейросеть принимает на вход действие и его результат, чтобы оценить функцию потерь и обновить веса сети.
reward_holder = tf.placeholder(shape=[1],dtype=tf.float32)
action_holder = tf.placeholder(shape=[1],dtype=tf.int32)
responsible_weight = tf.slice(weights,action_holder,[1])
loss = -(tf.log(responsible_weight)*reward_holder)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
update = optimizer.minimize(loss)

###Обучение агента

In [48]:
is_true = False
while(is_true == False):
    total_episodes = 1000 #Количество итераций обучения
    total_reward = np.zeros(num_bandits) #Изначальный выигрыш всех бандитов равен 0
    e = 0.1 #Вероятность случайного выбора
    init = tf.global_variables_initializer()

    #Запускаем граф tensorflow
    with tf.Session() as sess:
        sess.run(init)
        i = 0
        while i < total_episodes:
            
            #Выбираем действие либо случайно либо на основе нейросети
            if np.random.rand(1) < e:
                action = np.random.randint(num_bandits)
            else:
                action = sess.run(chosen_action)
            #Получаем результат игры, выбрав одного из бандитов
            reward = pullBandit(bandits[action]) 
            
            #Обновляем веса
            _,resp,ww = sess.run([update,responsible_weight,weights], 
                          feed_dict={reward_holder:[reward],action_holder:[action]})
            
            #Обновляем общий выигрыш каждого бандита
            total_reward[action] += reward
            # if i % 50 == 0:
            #     print("Общий выигрыш бандитов сейчас равен " + str(num_bandits) + 
            #     " bandits: " + str(total_reward))
            
            print("Значения коэффициентов: ", total_reward)
            print("Сумма значений(max 35)", np.sum(total_reward))
            #Катина попытка выжить начинается здесь
            if np.sum(total_reward)>=35:
              print("\n!FINAL! Сумма затрат на поставку: ", np.dot(total_reward, prices), "\n")
              break
            
            i+=1
    print("Агент думает, что бандит №" + str(np.argmax(ww)+1) + " идеален...")
    if np.argmax(ww) == np.argmax(-np.array(bandits)):
        print("...и он прав!")
        is_true = True
    else:
        print("...и он не прав!")
        is_true = False


Значения коэффициентов:  [-1.  0.  0.  0.]
Сумма значений(max 35) -1.0
Значения коэффициентов:  [-1.  1.  0.  0.]
Сумма значений(max 35) 0.0
Значения коэффициентов:  [-1.  2.  0.  0.]
Сумма значений(max 35) 1.0
Значения коэффициентов:  [-1.  3.  0.  0.]
Сумма значений(max 35) 2.0
Значения коэффициентов:  [-1.  4.  0.  0.]
Сумма значений(max 35) 3.0
Значения коэффициентов:  [-1.  5.  0.  0.]
Сумма значений(max 35) 4.0
Значения коэффициентов:  [-1.  6.  0.  0.]
Сумма значений(max 35) 5.0
Значения коэффициентов:  [-1.  5.  0.  0.]
Сумма значений(max 35) 4.0
Значения коэффициентов:  [-1.  6.  0.  0.]
Сумма значений(max 35) 5.0
Значения коэффициентов:  [-1.  7.  0.  0.]
Сумма значений(max 35) 6.0
Значения коэффициентов:  [-1.  8.  0.  0.]
Сумма значений(max 35) 7.0
Значения коэффициентов:  [-1.  7.  0.  0.]
Сумма значений(max 35) 6.0
Значения коэффициентов:  [-1.  8.  0.  0.]
Сумма значений(max 35) 7.0
Значения коэффициентов:  [-1.  9.  0.  0.]
Сумма значений(max 35) 8.0
Значения коэффициен

In [17]:
a = [1, 2, 3]
b = [1, 4, 3]
a<b

True